In [1]:
import polars as pl
import pandas as pd
import plotly.graph_objects as go
import utm
from filtering import Filtering

## Read in the Data

In [2]:
df = pl.scan_parquet(
 "/Users/max/Library/CloudStorage/Box-Box/Radar-Data/new_format/1678658102006.parquet"   
)

In [3]:
df = df.collect()

In [4]:
# create the file paths
network_outline_file = "./geo_data/network_outline.geojson"
radar_locations_file = "./geo_data/radar_origins.json"


f = Filtering(
    radar_location_path=radar_locations_file,
    network_boundary_path=network_outline_file,
)


In [5]:
df = (
    df
    # df.clone()
    .pipe(f.rotate_heading)
    .pipe(f.correct_center)
    .pipe(f.rotate_radars)
    .pipe(f.radar_to_utm)
    .pipe(f.radar_to_latlon)
    

    # .pipe(f.radar_to_h3)
    # .pipe(f.int_h3_2_str)
    # .pipe(f.filter_network_boundaries)
)


function: rotate_heading took: 1.016143798828125 seconds
function: correct_center took: 0.007606029510498047 seconds
function: rotate_radars took: 0.11610698699951172 seconds
function: radar_to_utm took: 0.22519397735595703 seconds
function: radar_to_latlon took: 0.12623310089111328 seconds


### Plot the Radars

In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import plotly.express as px

mapbox_key = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"
fig = go.Figure()

fig.update_layout(template="ggplot2", height=600, width=1200, font_size=18)


# create a color map for all the radar ips
radar_ips = df["ip"].unique().to_list()
radar_colors = px.colors.qualitative.D3
radar_color_map = {
    ip: radar_colors[i % len(radar_colors)] for i, ip in enumerate(radar_ips)
}

pandas_df = df.to_pandas()

plot_ips = [
    radar_ips[0],
]

for ip, r in pandas_df.groupby("ip"):
    if ip not in plot_ips:
        continue
    
    _s = r.sample(10000)

    fig.add_trace(
        go.Scattermapbox(
            lon=_s["lon_new"],
            lat=_s["lat_new"],
            mode="markers",
            opacity=1,
            line_width=4,
            name=ip,
            marker=dict(
                size=5,
                color="red",
                opacity=0.5,
            ),
            visible=True,
        )
    )

    fig.add_trace(
        go.Scattermapbox(
            lon=_s["lon"],
            lat=_s["lat"],
            mode="markers",
            opacity=1,
            line_width=4,
            name=ip,
            marker=dict(
                size=5,
                color=radar_color_map[ip],
                opacity=0.9,
            ),
            visible=True,
        )
    )

fig.update_layout(
    margin=go.layout.Margin(
        l=50,  # left margin
        r=50,  # right margin
        b=50,  # bottom margin
        t=50,  # top margin
    ),
    mapbox=dict(
        accesstoken=mapbox_key,
        bearing=0,
        # style as mapbox satellite
        style="satellite-streets",
        center=go.layout.mapbox.Center(lat=_s["lat"].mean(), lon=_s["lon"].mean()),
        pitch=0,
        zoom=14.1,
    ),
)


fig.show()


### Geolocate the Radar Data

In [ ]:
df.sort('epoch_time').groupby_dynamic(
    index_column='epoch_time',
    every='1s',
    by=['object_id'],
).agg([
    pl.col('x').mean().alias('x'),
    pl.col('y').mean().alias('y'),
    pl.col('f32_velocityInDir_mps').mean(),
    pl.col('ip').first()
]
)

In [ ]:
# create a unique index for the polars dataframe
pd_df = df.to_pandas()

In [ ]:
import plotly.express as px

# create a color map for all the radar ips
radar_ips = df['ip'].unique().to_list()
radar_colors = px.colors.qualitative.D3
radar_color_map = {ip : radar_colors[i % len(radar_colors)] for i, ip in enumerate(radar_ips)}

In [ ]:

pandas_df = df.to_pandas()

pandas_df['lat'], pandas_df['lon'] = utm.to_latlon(
        pandas_df['x'], pandas_df['y'], *radar_utm_zone
    )

In [ ]:
pandas_df.sample(10_000).to_csv('radar_data.csv', index=False)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

mapbox_key = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"
fig = go.Figure()

fig.update_layout(template="ggplot2", height=600, width=1200, font_size=18)


pandas_df = df.to_pandas()

pandas_df['lat'], pandas_df['lon'] = utm.to_latlon(
        pandas_df['x'], pandas_df['y'], *radar_utm_zone
    )

for ip, r in pandas_df.groupby("ip"):


    r['lat'], r['lon'] = utm.to_latlon(
        r['x'], r['y'], *radar_utm_zone
    )

    _s = r.sample(10000)
    fig.add_trace(
        go.Scattermapbox(
            lon=_s["lon"],
            lat=_s["lat"],
            mode="markers",
            opacity=1,
            line_width=4,
            name=ip,
            marker=dict(
                size=4,
                color=radar_color_map[ip],
                opacity=0.5,
            ),
            visible=True,
        )
    )

fig.update_layout(
    margin=go.layout.Margin(
        l=50,  # left margin
        r=50,  # right margin
        b=50,  # bottom margin
        t=50,  # top margin
    ),
    mapbox=dict(
        accesstoken=mapbox_key,
        bearing=0,
        # style as mapbox satellite
        style="satellite-streets",
        center=go.layout.mapbox.Center(lat=_s["lat"].mean(), lon=_s["lon"].mean()),
        pitch=0,
        zoom=14.1,
    ),
)


fig.show()


In [ ]:
df = df.with_columns([
    (pl.col('ui32_objectID').cast(str) + "_" + pl.col('ip')).alias('object_id'),
])

In [ ]:
df.groupby('object_id').agg(
    pl.col('epoch_time').diff().mean().alias('mean_dt'),
).to_pandas().mean_dt.dt.total_seconds().hist(bins=100)


In [ ]:
df.filter(pl.col('object_id').is_in([ids]))

In [ ]:
tmp = df.filter(pl.col('object_id').is_in([ids])).to_pandas()

In [ ]:
tmp.plot.line(x='epoch_time', y='f32_velocityInDir_mps')

In [ ]:
# # plot a couple individual objects
# ids = df.sample(1).select('object_id').to_numpy().flatten()[0]

# # set the plot method to plotly
# pd.set_option("plotting.backend", "plotly")

df.filter(pl.col('object_id').is_in([ids])).to_pandas().sort_values('epoch_time').plot.line('epoch_time', 'f32_velocityInDir_mps',)

### Plot Distance to TL1 Stop Bar

In [ ]:
TL1_POS = (33.233039472986604, -87.62266063800959)
TL1_POS_UTM = utm.from_latlon(*TL1_POS, force_zone_number=radar_utm_zone[0] ,force_zone_letter=radar_utm_zone[1])[:2]


In [ ]:
df = df.with_columns([
    (((pl.col('x') - TL1_POS_UTM[0]) ** 2 + (pl.col('y') - TL1_POS_UTM[1]) ** 2) ** 0.5).alias('distance_from_tl1'),
])

In [ ]:
df = df.sort('epoch_time')

# only consider objects that move closer to TL1 during their time in the network
keep_ids = df.groupby('object_id').agg([
    pl.col('distance_from_tl1').first().alias('start_distance'),
    pl.col('distance_from_tl1').last().alias('end_distance'),
]
).filter(
    pl.col('start_distance') > pl.col('end_distance')
).select('object_id').to_pandas().object_id.values

In [ ]:
wb_df = df.filter(pl.col('object_id').is_in(keep_ids.tolist()))
wb_df = wb_df.to_pandas()
# wb_df.epoch_time.min(), wb_df.epoch_time.max()
# only look at the first two minutes of data
wb_df = wb_df[wb_df.epoch_time < wb_df.epoch_time.max() - pd.Timedelta('60 seconds')]

In [ ]:
fig = go.Figure()

keep_ips = ['10.160.7.136', '10.160.7.137', '10.160.7.141', '10.160.7.142', '10.160.7.146', '10.160.7.147']
# keep_ips = '10.160.7.142', '10.160.7.146'

plotted_ips = set()
for veh in wb_df.object_id.unique():

    _df = wb_df[wb_df.object_id == veh]

    if _df.ip.iloc[0] not in keep_ips:
        continue

    if _df.ip.iloc[0] in plotted_ips:
        show_legend = False
    else:
        show_legend = True
        plotted_ips.add(_df.ip.iloc[0])

    fig.add_trace(
        go.Scatter(
            x=_df.epoch_time,
            y=_df.distance_from_tl1,
            mode="lines",
            opacity=1,
            line_color=radar_color_map[_df.ip.iloc[0]],
            name=_df.ip.iloc[0],
            showlegend=show_legend,
        )
    )

# update the height and width of the figure
fig.update_layout(
    height=800,
    width=1000,
    font_size=18,
)


fig.show()

### Investigate the Radar Data

In [ ]:
df.filter(pl.col('object_id').is_in(['46671_10.160.7.137']))

In [ ]:
veh_df = df.filter(pl.col('object_id').is_in(['46671_10.160.7.137'])).to_pandas()

fig = veh_df.plot.line('epoch_time', 'f32_velocityInDir_mps', )

# differentiate the position to get velocity
veh_df['distance'] = (veh_df['f32_positionX_m'].diff() ** 2 + veh_df['f32_positionY_m'].diff() ** 2) ** 0.5
veh_df['velocity'] = veh_df['distance'] / veh_df['epoch_time'].diff().dt.total_seconds()

fig.add_trace(
    go.Scatter(
        x=veh_df.epoch_time,
        y=veh_df.velocity,
        mode="lines",
    )
)

fig.show()

### Plot Via Position

In [ ]:
# keep_ips = ['10.160.7.136', '10.160.7.137', '10.160.7.141', '10.160.7.142', '10.160.7.146']
keep_ips = '10.160.7.142', '10.160.7.146'

mapbox_key = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"
fig = go.Figure()
fig.update_layout(template="ggplot2", height=600, width=1200, font_size=18)

plotted_ips = set()

times = []
unique_times = set()
plotted_ips = set()

for veh in wb_df.object_id.unique():

    _df = wb_df[wb_df.object_id == veh].copy()

    _df['lat'], _df['lon'] = utm.to_latlon(
        _df['x'], _df['y'], *radar_utm_zone
    )

    ip = _df.ip.iloc[0]

    if ip not in keep_ips:
        continue
    # resample the data to 1 second intervals
    _df = _df.set_index('epoch_time').resample('5s').mean(numeric_only=True).reset_index()
    

    for time in _df.epoch_time.unique():
        
        times.append(time)
        unique_times.add(time)

        # filter to the current time
        _df = _df[_df.epoch_time <= time]

        if ip in plotted_ips:
            show_legend = False
        else:
            show_legend = True
            plotted_ips.add(ip)
                

        fig.add_trace(
            go.Scattermapbox(
                lon=_df['lon'],
                lat=_df['lat'],
                mode="markers",
                opacity=1,
                marker_color=radar_color_map[ip],
                name=ip,
                showlegend=show_legend,
                visible=show_legend,
            )
        )


# # create a plotly slider to change the angle
# steps = []
# for i, time_ in enumerate(unique_times):
#     step = dict(
#         method="update",
#         args=[{"visible": [False] * len(fig.data)},
#               {"title": "Slider switched to step: " + str(time_)}],  # layout attribute
#     )
#     step["args"][0]["visible"] = [True if t < time_ else False for t in times]
#     steps.append(step)

# sliders = [
#     dict(active=5, currentvalue={"prefix": ""}, pad={"t": 50}, steps=steps)
# ]

# fig.update_layout(sliders=sliders)
fig.update_layout(
    margin=go.layout.Margin(
        l=50,  # left margin
        r=50,  # right margin
        b=50,  # bottom margin
        t=50,  # top margin
    ),
    mapbox=dict(
        accesstoken=mapbox_key,
        bearing=0,
        # style as mapbox satellite
        style="satellite-streets",
        center=go.layout.mapbox.Center(lat=_s["lat"].mean(), lon=_s["lon"].mean()),
        pitch=0,
        zoom=14.1,
    ),
)
fig.show()